In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os
from time import sleep
import win32api
import win32con
import win32clipboard
import ctypes
import re
import datetime

In [ ]:
def click(x, y):
    win32api.SetCursorPos((x, y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, x, y, 0, 0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, x, y, 0, 0)

# Save the Contents of Each PDF to a Text File

In [ ]:
# Due to differences in PDF formatting, the most consistent way to extract text from these pdfs
# is to open the PDF, select all of the text, and save it to a file

fileLocation = "./PDFs"

driver = webdriver.Chrome(ChromeDriverManager().install())

# clear clipboard
win32clipboard.OpenClipboard()
win32clipboard.EmptyClipboard()
win32clipboard.CloseClipboard()

outputLocation = fileLocation + "/Text/"
if not os.path.exists(outputLocation):
    os.makedirs(outputLocation)


for filename in os.listdir(fileLocation):
    if filename.endswith(".pdf"):
        fullPath = fileLocation + "/" + filename
        outputLocation = fileLocation + "/Text/" + filename.replace(".pdf", ".txt")

        driver.get(fullPath)
        sleep(2)
        click(150, 450)
        sleep(0.25)
        # CTRL + A
        ctypes.windll.user32.keybd_event(
            0x11, 0, 0, 0)  # 0x11  CTRL key down
        sleep(0.25)
        ctypes.windll.user32.keybd_event(
            0x41, 0, 0, 0)  # 0x41 A key down
        sleep(0.25)
        ctypes.windll.user32.keybd_event(
            0x41, 0, 0x0002, 0)  # 0x41 A key up
        sleep(0.25)
        ctypes.windll.user32.keybd_event(
            0x11, 0, 0x0002, 0)  # 0x11  CTRL key up
        sleep(3)
        # CTRL + C
        ctypes.windll.user32.keybd_event(
            0x11, 0, 0, 0)  # 0x11  CTRL key down
        sleep(0.25)
        ctypes.windll.user32.keybd_event(
            0x43, 0, 0, 0)  # 0x43 C key down
        sleep(0.25)
        ctypes.windll.user32.keybd_event(
            0x43, 0, 0x0002, 0)  # 0x43 C key up
        sleep(0.25)
        ctypes.windll.user32.keybd_event(
            0x11, 0, 0x0002, 0)  # 0x11  CTRL key up
        sleep(3)
        win32clipboard.OpenClipboard()
        data = win32clipboard.GetClipboardData()
        win32clipboard.EmptyClipboard()
        win32clipboard.CloseClipboard()
        f = open(outputLocation, "w")
        f.write(data)
        f.close()

driver.quit()

# Basic Formatting of the Text Files

In [ ]:
# format the text files

filePath = './PDFs/Text'
formatFilePath = './PDFs/Text/Extracted'

if not os.path.exists(formatFilePath):
    os.makedirs(formatFilePath)

sections = []

for fileName in os.listdir(filePath):
    if fileName.endswith('.txt'):
        multiFile = False
        date = ''
        fullPath = filePath + '/' + fileName
        f = open(fullPath,'r')
        lines = []
        strangeFormat = ''
        dispLine = False
        
        for line in f:
            if not('Brockton Police Department Press Log Page:' in line or line.strip() == ""):
                # Determin how many days this file looks at
                if 'Dispatch Log From:' in line:
                    dispLine = True
                    parts = line.split(':')
                    fromDate = parts[1].strip().split(' ')[0].split('/')
                    fromDate = datetime.datetime(int(fromDate[2]),int(fromDate[0]),int(fromDate[1]))
                    
                    thruDate = parts[2].strip().split(' ')[0].split('/')
                    thruDate = datetime.datetime(int(thruDate[2]),int(thruDate[0]),int(thruDate[1]))
                    
                    dif = thruDate - fromDate
                    dif = dif.days
                    
                    if dif > 1:
                        multiFile = True
                    
                elif 'Narrative:' in line:
                    # A few files contain a different format than the rest, these can be ignored
                    strangeFormat = 1
                elif 'For Date:' in line:
                    # Get the date that the file is referencing
                    if date == '':
                        date = line.split(':')[1].split('-')[0].replace('/','.').replace('\n','').strip()
                    elif multiFile:
                        sections.append({'Date':date,'Lines':lines.copy()})
                        date = line.split(':')[1].split('-')[0].replace('/','.').replace('\n','').strip()
                        lines = []
                    else:
                        # if the file contains records from the next file, do not 
                        # include the lines as they will be recorded later
                        break
                        
                if not dispLine:
                    lines.append(line)
                else:
                    dispLine = False
                
        f.close()
        
        # Some files store a month of data, 
        # break these down into individual days
        if multiFile:
            sections.append({'Date':date,'Lines':lines.copy()})
            
            for section in sections:
                outputPath = formatFilePath + '/' + section['Date'] + '.txt'
                
                if strangeFormat:
                    print('File contains different format',date)
                    
                elif not os.path.isfile(outputPath):
                    print('Creating new file', outputPath)
                    nf = open(outputPath,'w')
                    for line in section['Lines']:
                        nf.write(line)
                    nf.close()
        
        else:
            
            outputPath = formatFilePath + '/' + date + '.txt'
            
            if strangeFormat:
                print('File contains different format',date)

            elif not os.path.isfile(outputPath):
                print('Creating new file', outputPath)
                nf = open(outputPath,'w')
                for line in lines:
                    nf.write(line)
                nf.close()

# Add Call Number Column to January - March 2015 

In [ ]:
fp = './PDFs/Text/Extracted'
for fileName in os.listdir(fp):
    if fileName.endswith('-New.txt'):
        fullPath = fp + '/' + fileName
        os.remove(fullPath)

In [ ]:
# Fix missing Call numbers for Jan 2015 - March 2015

fp = './PDFs/Text/Extracted'
newNumber = 0
for fileName in os.listdir(fp):
    if fileName.endswith('.txt'):
        date = fileName.split('.')
        year = date[2]
        month = date[0]
        if int(year) == 2015 and int(month) < 4:
            fullPath = fp + '/' + fileName
            newFile = fp + '/' + fileName.strip('.txt') + '-New.txt'
            f = open(fullPath,'r')
            nf = open(newFile, 'w')
            
            for line in f:
                if 'Time' in line and 'Call' in line and 'Reason' in line and 'Action' in line:
                    pass
                elif re.search("^ \d{4} ", line):
                    newNumber += 1
                    newCallNumber = '15-'+str(newNumber)
                    newLine = newCallNumber + line
                    nf.write(newLine)
                else:
                    nf.write(line)
            
            f.close()
            nf.close()

In [ ]:
toRemove = []

fp = './PDFs/Text/Extracted'

for fileName in os.listdir(fp):
    if fileName.endswith('-New.txt'):
        fn = fileName.strip('-New.txt') + '.txt'
        fullPath = fp + '/' + fn
        toRemove.append(fullPath)

# remove old files
for filePath in toRemove:
    os.remove(filePath)

# rename new file to old file
for filePath in toRemove:
    basePath = filePath.split('/')[:-1]
    basePath = '/'.join(basePath)
    correctName = basePath + '/' + (filePath.split('/')[-1])
    oldName = basePath + '/' + filePath.split('/')[-1].split('.txt')[0] + '-New.txt'
    os.rename(oldName,correctName)

# Check for Missing Days

In [ ]:
# Check for missing files

totalMissingDays = 0

fp = '.PDFs/Text/Extracted'

dates = []

for file in os.listdir(fp):
    if file.endswith('.txt'):
        dates.append(file.replace('.txt',''))

dates = list(set(dates))

# Sort by YearMonthDay
dates = sorted(dates,key=lambda x: int(x.split('.')[2]+x.split('.')[0]+x.split('.')[1]))

# Get the first date
firstDate = dates[0]
firstDate = firstDate.split('.')

compDate = datetime.datetime(int(firstDate[2]),int(firstDate[0]),int(firstDate[1])) 

# Increment the date by one and check if it matches the next date in the list
# If not, that date might be missing and should be looked for in another location
# on http://www.brocktonpolice.com/
for date in dates:
    currentDate = date.split('.')
    currentDate = datetime.datetime(int(currentDate[2]),int(currentDate[0]),int(currentDate[1]))
    if currentDate != compDate:
        dif = currentDate - compDate
        dif = dif.days
        # If the next date found is several days away, print all of the missing days
        # between the tow dates
        if dif > 1:
            for i in range(0,dif):
                print('You may be missing',compDate.strftime('%m/%d/%Y'))
                compDate += datetime.timedelta(days=1)
                totalMissingDays += 1
            compDate = currentDate
            compDate += datetime.timedelta(days=1)
        else:
            print('You may be missing',compDate.strftime('%m/%d/%Y'))
            totalMissingDays += 1
            compDate = currentDate
            compDate += datetime.timedelta(days=1)
    else:
        compDate += datetime.timedelta(days=1)
print('Total Missing Days:',totalMissingDays)